In [1]:
#!pip3 install -q -U google-genai

In [2]:
import dotenv
import os
import modules.tools as tools
dotenv.load_dotenv(dotenv_path=".env")

True

In [4]:
#We can use "gpt-3.5-turbo" or "gemini-2.5-flash", #Uncomment if using Google GenAI

#Let's declare our sub agent instances first.
MyMarketResearcher = MarketResearchAgent(model="gemini-2.5-flash")
MyNewsResearcher = MarketSentimentAgent(model="gemini-2.5-flash")
MyWriter = WriterAgent(model="gemini-2.5-flash")

#We put all of our sub agents together as a list of objects
MyAgentsTeam = {MyMarketResearcher, MyNewsResearcher, MyWriter}

#Now, we declare our main orchestrator agent instance.
MyOrchestrator = OrchestratorAgent(model="gpt-3.5-turbo", agents=MyAgentsTeam)

{'Yahoo Finance Stock Quote': <modules.tools.YahooFinance at 0x1731c758ec0>,
 'Financial Modeling Prep Quote': <modules.tools.FMP at 0x1730bf5e3c0>,
 'FinnHub News': <modules.tools.FinnHub at 0x1731c9376e0>}

In [5]:
# Import libraries for beautiful HTML display in Jupyter notebook
from IPython.display import HTML, display, clear_output
from datetime import datetime
import time

You are a helpful Finance assistant that explains things in a few words.
          You have the following tools:
            [{'name': 'Yahoo Finance Stock Quote', 'description': 'Get the latest stock quote for a given symbol from Yahoo Finance.', 'api': '{ ""symbol": "AAPL"}'}, {'name': 'Financial Modeling Prep Quote', 'description': 'Get the latest stock quote for a given symbol from Financial Modeling Prep (FMP).', 'api': '{ ""symbol": "AAPL"}'}, {'name': 'FinnHub News', 'description': 'Get the latest financial news for a given symbol from FinnHub.', 'api': '{ ""symbol": "AAPL"}'}]
            
            You should only use the tools listed above.
            When you use a tool, you must call the API exactly as shown above.
            You must always include the symbol in the API call.
            You must always include the step in the API call.
            You must always use double quotes for the JSON keys and string values in the API call.
            You must never use sing

In [6]:
# CSS styles for beautiful chatbot interface
chatbot_css = """
<style>
.chat-container {
    max-width: 800px;
    margin: 0 auto;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    border-radius: 20px;
    padding: 20px;
    box-shadow: 0 10px 30px rgba(0,0,0,0.3);
}

.chat-header {
    text-align: center;
    color: white;
    font-size: 24px;
    font-weight: bold;
    margin-bottom: 20px;
    text-shadow: 2px 2px 4px rgba(0,0,0,0.5);
}

.message {
    margin: 15px 0;
    display: flex;
    align-items: flex-start;
}

.user-message {
    justify-content: flex-end;
}

.bot-message {
    justify-content: flex-start;
}

.message-bubble {
    max-width: 70%;
    padding: 15px 20px;
    border-radius: 20px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.15);
    position: relative;
    line-height: 1.4;
}

.user-bubble {
    background: linear-gradient(135deg, #36d1dc 0%, #5b86e5 100%);
    color: white;
    margin-left: 30px;
}

.bot-bubble {
    background: white;
    color: #333;
    margin-right: 30px;
    border: 1px solid #e0e0e0;
}

.message-avatar {
    width: 40px;
    height: 40px;
    border-radius: 50%;
    margin: 0 10px;
    display: flex;
    align-items: center;
    justify-content: center;
    font-weight: bold;
    color: white;
    font-size: 16px;
}

.user-avatar {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
}

.bot-avatar {
    background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
}

.timestamp {
    font-size: 11px;
    color: rgba(255,255,255,0.7);
    margin-top: 5px;
    text-align: right;
}

.bot-timestamp {
    color: #999;
    text-align: left;
}

.typing-indicator {
    display: flex;
    align-items: center;
    justify-content: flex-start;
    margin: 15px 0;
}

.typing-bubble {
    background: white;
    padding: 15px 20px;
    border-radius: 20px;
    margin-right: 30px;
    margin-left: 50px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.15);
}

.typing-dots {
    display: flex;
    gap: 4px;
}

.dot {
    width: 8px;
    height: 8px;
    background: #999;
    border-radius: 50%;
    animation: typing 1.4s infinite ease-in-out;
}

.dot:nth-child(1) { animation-delay: -0.32s; }
.dot:nth-child(2) { animation-delay: -0.16s; }

@keyframes typing {
    0%, 80%, 100% { transform: scale(0.8); opacity: 0.5; }
    40% { transform: scale(1); opacity: 1; }
}

.input-container {
    margin-top: 20px;
    text-align: center;
}

.status-message {
    text-align: center;
    color: white;
    font-style: italic;
    margin: 10px 0;
    opacity: 0.8;
}
</style>
"""

# Helper functions for chatbot display
def display_chat_header():
    """Display the chatbot header"""
    header_html = f"""
    {chatbot_css}
    <div class="chat-container">
        <div class="chat-header">
            💰 CapitalMind Financial Assistant 🤖
        </div>
        <div class="status-message">
            Welcome! Ask me anything about financial markets and stocks.
        </div>
    </div>
    """
    display(HTML(header_html))

def display_user_message(message):
    """Display user message with styling"""
    timestamp = datetime.now().strftime("%H:%M")
    user_html = f"""
    <div class="chat-container">
        <div class="message user-message">
            <div class="message-bubble user-bubble">
                {message}
                <div class="timestamp">{timestamp}</div>
            </div>
            <div class="message-avatar user-avatar">👤</div>
        </div>
    </div>
    """
    display(HTML(user_html))

def display_typing_indicator():
    """Display typing indicator"""
    typing_html = f"""
    <div class="chat-container">
        <div class="typing-indicator">
            <div class="message-avatar bot-avatar">🤖</div>
            <div class="typing-bubble">
                <div class="typing-dots">
                    <div class="dot"></div>
                    <div class="dot"></div>
                    <div class="dot"></div>
                </div>
            </div>
        </div>
    </div>
    """
    display(HTML(typing_html))

def display_bot_message(message):
    """Display bot message with styling"""
    timestamp = datetime.now().strftime("%H:%M")
    # Format the message with line breaks for better readability
    formatted_message = message.replace('\n', '<br>')
    
    bot_html = f"""
    <div class="chat-container">
        <div class="message bot-message">
            <div class="message-avatar bot-avatar">🤖</div>
            <div class="message-bubble bot-bubble">
                {formatted_message}
                <div class="timestamp bot-timestamp">{timestamp}</div>
            </div>
        </div>
    </div>
    """
    display(HTML(bot_html))

def display_status_message(message):
    """Display status message"""
    status_html = f"""
    <div class="chat-container">
        <div class="status-message">
            {message}
        </div>
    </div>
    """
    display(HTML(status_html))

def clear_typing_indicator():
    """Clear the typing indicator"""
    clear_output(wait=True)

In [12]:
#Displaying the header and launching the chat bot.
display_chat_header()

## Create a conversation loop with user input and orchestrator response
conversation_history = []

while True:
    try:
        user_input = input("💬 Your question: ")
        
        # Display user message with beautiful styling
        display_user_message(user_input)
        conversation_history.append(f"User: {user_input}")
        
        if user_input.lower() == 'exit':
            display_status_message("👋 Exiting the financial market assistant. Goodbye!")
            break
        
        # Show typing indicator while processing
        display_typing_indicator()
        time.sleep(1)  # Brief pause for realistic effect
        
        # Get response from orchestrator
        response = MyOrchestrator.reAct(user_input)  # Pass just the user input, not the full history
        conversation_history.append(f"Orchestrator: {response}")
        
        # Clear typing indicator and display bot response
        clear_output(wait=True)
        display_chat_header()
        
        # Redisplay recent conversation
        recent_history = conversation_history[-6:]  # Show last 3 exchanges
        for i in range(0, len(recent_history), 2):
            if i < len(recent_history):
                # Display user message
                user_msg = recent_history[i].replace("User: ", "")
                display_user_message(user_msg)
            if i + 1 < len(recent_history):
                # Display bot message
                bot_msg = recent_history[i + 1].replace("Orchestrator: ", "")
                display_bot_message(bot_msg)
        
    except KeyboardInterrupt:
        display_status_message("👋 Chat interrupted. Goodbye!")
        break
    except Exception as e:
        display_bot_message(f"❌ Sorry, I encountered an error: {str(e)}")
        display_status_message("Please try asking your question again.")
    

User:  Summarize the latest news from Apple and give me today's stock price.
Need to Call FinnHub News. Type Y/y to continue...) y


Invoking FinnHub News with arguments {'symbol': 'AAPL', 'step': 1}


Need to Call Yahoo Finance Stock Quote. Type Y/y to continue...) y


Invoking Yahoo Finance Stock Quote with arguments {'symbol': 'AAPL', 'step': 2}
Final Answer: Apple is deeply involved in AI, planning smart glasses and integrating AI into its products. The iPhone 17 is seeing strong demand, though some analysts are cautious about future models' expectations. Apple faces legal scrutiny regarding chip royalties and data collection. The stock price is **$254.04**.


Do you want to exit? Type Y/y to exit... y


Thanks for chatting! Goodbye!
